In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
train_df = pd.read_csv('../../final/data/train_set_noisy_2_missing_3.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,8.839534,485.156879,-1.000000,0.000000,453.705337,102.432354,-1.000000,2.588889,1,1.452379,115.041719,115.082449,-1.0,-1.000000,-1.00000,-1.000000,-1.000000,0.0
1,10.306951,-1.000000,-1.000000,3.269642,361.637557,101.949912,213.802158,3.032946,0,1.983531,3.371683,128.512264,-1.0,19.486851,133.03569,30.920853,59.120563,1.0
2,6.488013,80.274764,4.562126,4.233714,116.337191,77.768892,-1.000000,2.502805,0,-1.000000,74.749421,-1.000000,-1.0,-1.000000,-1.00000,-1.000000,-1.000000,3.0
3,14.955260,485.921046,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.00000,-1.000000,-1.000000,0.0
4,7.098801,-1.000000,-1.000000,-1.000000,495.647437,-1.000000,-1.000000,-1.000000,0,0.395440,-1.000000,-1.000000,-1.0,-1.000000,-1.00000,21.296403,-1.000000,1.0


In [4]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [5]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [6]:
train_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [7]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [8]:
for steps in [int(13.5e6), int(14e6)]:
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../../final/models/dqn_noisy_2_missing_3_{steps}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.92     |
|    ep_rew_mean      | -0.8     |
|    exploration_rate | 0.801    |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 1813     |
|    time_elapsed     | 155      |
|    total_timesteps  | 282675   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.03     |
|    n_updates        | 58168    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.4      |
|    ep_rew_mean      | -0.88    |
|    exploration_rate | 0.58     |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 1427     |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.59     |
|    ep_rew_mean      | -0.26    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.42     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 1052     |
|    time_elapsed     | 4935     |
|    total_timesteps  | 5194213  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.151    |
|    n_updates        | 1286053  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.24     |
|    ep_rew_mean      | -0.18    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.46     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 1045     |
|    time_elapsed     | 5286     |
|    total_timesteps  | 5527207  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.6      |
|    ep_rew_mean      | -0.38    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.35     |
| time/               |          |
|    episodes         | 3100000  |
|    fps              | 1001     |
|    time_elapsed     | 10545    |
|    total_timesteps  | 10557712 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.178    |
|    n_updates        | 2626927  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.41     |
|    ep_rew_mean      | -0.28    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.42     |
| time/               |          |
|    episodes         | 3200000  |
|    fps              | 991      |
|    time_elapsed     | 11006    |
|    total_timesteps  | 10908469 |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.16     |
|    ep_rew_mean      | -0.98    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.09     |
| time/               |          |
|    episodes         | 700000   |
|    fps              | 1163     |
|    time_elapsed     | 1873     |
|    total_timesteps  | 2179705  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 3.44     |
|    n_updates        | 532426   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.76     |
|    ep_rew_mean      | -0.84    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 800000   |
|    fps              | 1143     |
|    time_elapsed     | 2200     |
|    total_timesteps  | 2516992  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.75     |
|    ep_rew_mean      | -0.38    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.32     |
| time/               |          |
|    episodes         | 2200000  |
|    fps              | 1050     |
|    time_elapsed     | 6862     |
|    total_timesteps  | 7209064  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.162    |
|    n_updates        | 1789765  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.46     |
|    ep_rew_mean      | -0.34    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.36     |
| time/               |          |
|    episodes         | 2300000  |
|    fps              | 1047     |
|    time_elapsed     | 7212     |
|    total_timesteps  | 7552338  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.66     |
|    ep_rew_mean      | -0.24    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.42     |
| time/               |          |
|    episodes         | 3700000  |
|    fps              | 962      |
|    time_elapsed     | 12926    |
|    total_timesteps  | 12445485 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.139    |
|    n_updates        | 3098871  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.66     |
|    ep_rew_mean      | -0.18    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.44     |
| time/               |          |
|    episodes         | 3800000  |
|    fps              | 959      |
|    time_elapsed     | 13347    |
|    total_timesteps  | 12807047 |
| train/              |          |
|    learning_rate  

In [9]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_4_missing_1_7500000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [10]:
#utils.diagnose_sample(dqn_model, X_test, y_test, 1)

In [ ]:
# test_df[(test_df.y_pred==1) & (test_df.y_actual==1)]

In [ ]:
test_df.y_pred.value_counts()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_1_11000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_1_11000000.csv', index=False)